In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import os
import glob
import sys

In [2]:
from utils import set_basepath

In [3]:
sys.path[0]

'/Users/jacobhow/Documents/programming-projects/nfl-big-data/notebooks/feature'

In [4]:
basepath = set_basepath(steps=2)

In [5]:
sys.path[0]

'/Users/jacobhow/Documents/programming-projects/nfl-big-data'

In [6]:
from src.team import Team
from src.game import Game
from src.play import Play
from src.player import Player
from src.analysis import process_games, aggregate_coverages

In [7]:
games = pd.read_csv(os.path.join(basepath,'data/games.csv'))

In [8]:
players = pd.read_csv(os.path.join(basepath,'data/players.csv')).set_index('nflId')

In [9]:
plays = pd.read_csv(os.path.join(basepath,'data/plays.csv'))

In [10]:
weeks = {}
for i in range(17):
    name = f'week{i+1}'
    weeks[name] = pd.read_csv(os.path.join(basepath,'data',name+'-processed.csv'))

In [11]:
teams = process_games(games,plays)

In [12]:
team = teams['IND']

In [13]:
team.games

{'week1': <src.game.Game at 0x1a29c89630>,
 'week2': <src.game.Game at 0x1a2f15e3c8>,
 'week3': <src.game.Game at 0x107ee7128>,
 'week4': <src.game.Game at 0x107ef1ef0>,
 'week5': <src.game.Game at 0x107f09e48>,
 'week6': <src.game.Game at 0x108c1bba8>,
 'week7': <src.game.Game at 0x108c2e470>,
 'week8': <src.game.Game at 0x108c48ba8>,
 'week10': <src.game.Game at 0x1a1d34a9b0>,
 'week11': <src.game.Game at 0x1a1d35eef0>,
 'week12': <src.game.Game at 0x1a1d378470>,
 'week13': <src.game.Game at 0x1a1d394668>,
 'week14': <src.game.Game at 0x1a1d3a4908>,
 'week15': <src.game.Game at 0x1a1d3c0710>,
 'week16': <src.game.Game at 0x1a1d3cac50>,
 'week17': <src.game.Game at 0x1a1d5aeba8>}

In [14]:
team.process_weeks(weeks)

week1


/Users/jacobhow/Documents/programming-projects/nfl-big-data/src/play.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.player_tracking['distance from line'] = self.player_tracking['x'] - self.line_of_scrimmage
/Users/jacobhow/Documents/programming-projects/nfl-big-data/src/play.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.player_tracking['distance to sideline'] = [min((160/3) - y,y) for y in self.player_tracking['y'].values]
/Users/jacobhow/Documents/programming-projects/nfl-big-data/

week2
week3
week4
week5
week6
week7
week8
week9
week10
week11
week12
week13
week14
week15
week16
week17


In [15]:
for game in team.games.values():
    game.process_plays(players)

Play Error
Game ID: 2018121605 - Play ID: 1770
(2:34) (Shotgun) D.Prescott pass incomplete deep left to N.Brown. IND-Q.Wilson was injured during the play.
Play Error
Game ID: 2018121605 - Play ID: 2544
(11:23) D.Prescott pass short right to N.Brown to DAL 39 for 14 yards (M.Adams, A.Walker).


In [16]:
game_coverages = []

for game in team.games.values():
    _coverage = game.classify_defensive_back_coverages(useId=True)
    game_coverages.append(_coverage)

check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
chec

In [17]:
season_coverages = aggregate_coverages(coverages=game_coverages)

In [18]:
season_coverages

{2557961: {'snaps': 259,
  'zone': 25,
  'zone-deep': 52,
  'zone-over': 75,
  'man': 106,
  'man-over': 0,
  'blitz': 1,
  'games played': 13},
 2558108: {'snaps': 210,
  'zone': 23,
  'zone-deep': 29,
  'zone-over': 63,
  'man': 91,
  'man-over': 0,
  'blitz': 4,
  'games played': 10},
 2558976: {'snaps': 472,
  'zone': 24,
  'zone-deep': 14,
  'zone-over': 216,
  'man': 172,
  'man-over': 0,
  'blitz': 46,
  'games played': 15},
 2560789: {'snaps': 487,
  'zone': 40,
  'zone-deep': 1,
  'zone-over': 369,
  'man': 41,
  'man-over': 0,
  'blitz': 36,
  'games played': 15},
 2560816: {'snaps': 30,
  'zone': 3,
  'zone-deep': 0,
  'zone-over': 25,
  'man': 0,
  'man-over': 0,
  'blitz': 2,
  'games played': 5},
 2552386: {'snaps': 367,
  'zone': 79,
  'zone-deep': 183,
  'zone-over': 68,
  'man': 28,
  'man-over': 0,
  'blitz': 9,
  'games played': 12},
 2558060: {'snaps': 474,
  'zone': 29,
  'zone-deep': 439,
  'zone-over': 5,
  'man': 1,
  'man-over': 0,
  'blitz': 0,
  'games played

In [19]:
df = pd.DataFrame.from_dict(season_coverages, orient='index')

In [20]:
df

,snaps,zone,zone-deep,zone-over,man,man-over,blitz,games played
2557961,259,25,52,75,106,0,1,13
2558108,210,23,29,63,91,0,4,10
2558976,472,24,14,216,172,0,46,15
2560789,487,40,1,369,41,0,36,15
2560816,30,3,0,25,0,0,2,5
2552386,367,79,183,68,28,0,9,12
2558060,474,29,439,5,1,0,0,14
2561155,88,4,1,68,5,0,10,11
2556429,93,16,52,16,7,0,2,5
2558851,329,18,4,268,10,0,29,15


In [21]:
players.head()

,height,weight,birthDate,collegeName,position,displayName
nflId,,,,,,
2539334,72,190,1990-09-10,Washington,CB,Desmond Trufant
2539653,70,186,1988-11-01,Southeastern Louisiana,CB,Robert Alford
2543850,69,186,1991-12-18,Purdue,SS,Ricardo Allen
2555162,73,227,1994-11-04,Louisiana State,MLB,Deion Jones
2555255,75,232,1993-07-01,Minnesota,OLB,De'Vondre Campbell


In [22]:
df = pd.merge(df,players[['displayName','position']],left_index=True,right_index=True)

In [23]:
df

,snaps,zone,zone-deep,zone-over,man,man-over,blitz,games played,displayName,position
2557961,259,25,52,75,106,0,1,13,Quincy Wilson,CB
2558108,210,23,29,63,91,0,4,10,Nate Hairston,CB
2558976,472,24,14,216,172,0,46,15,Kenny Moore II,CB
2560789,487,40,1,369,41,0,36,15,Darius Leonard,LB
2560816,30,3,0,25,0,0,2,5,Skai Moore,ILB
2552386,367,79,183,68,28,0,9,12,Clayton Geathers,FS
2558060,474,29,439,5,1,0,0,14,Malik Hooker,FS
2561155,88,4,1,68,5,0,10,11,Zaire Franklin,ILB
2556429,93,16,52,16,7,0,2,5,Matthias Farley,SS
2558851,329,18,4,268,10,0,29,15,Anthony Walker,ILB


In [24]:
coverage_names = ('zone','zone-deep','zone-over','man','man-over','blitz')

for key in coverage_names:
    df[key] /= df['snaps']

In [25]:
df

,snaps,zone,zone-deep,zone-over,man,man-over,blitz,games played,displayName,position
2557961,259,0.096525,0.200772,0.289575,0.409266,0.0,0.003861,13,Quincy Wilson,CB
2558108,210,0.109524,0.138095,0.300000,0.433333,0.0,0.019048,10,Nate Hairston,CB
2558976,472,0.050847,0.029661,0.457627,0.364407,0.0,0.097458,15,Kenny Moore II,CB
2560789,487,0.082136,0.002053,0.757700,0.084189,0.0,0.073922,15,Darius Leonard,LB
2560816,30,0.100000,0.000000,0.833333,0.000000,0.0,0.066667,5,Skai Moore,ILB
2552386,367,0.215259,0.498638,0.185286,0.076294,0.0,0.024523,12,Clayton Geathers,FS
2558060,474,0.061181,0.926160,0.010549,0.002110,0.0,0.000000,14,Malik Hooker,FS
2561155,88,0.045455,0.011364,0.772727,0.056818,0.0,0.113636,11,Zaire Franklin,ILB
2556429,93,0.172043,0.559140,0.172043,0.075269,0.0,0.021505,5,Matthias Farley,SS
2558851,329,0.054711,0.012158,0.814590,0.030395,0.0,0.088146,15,Anthony Walker,ILB
